In [25]:
# Hospital Patient Data Analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [26]:
#1 Load the patient dataset and show summary with info().
patient_df = pd.read_csv("Patient_Data.csv")
billing_df = pd.read_csv("Billing_Data.csv")

In [27]:
print("Patient Data Info:")
patient_df.info()

Patient Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [28]:
print("Billing Data Info:")
billing_df.info()

Billing Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [29]:
#2 Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_billing_df = patient_df[billing_cols]

In [30]:
#3 Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
admin_cols = ['ReceptionistID', 'CheckInTime']
patient_df_cleaned = patient_df.drop(columns=[c for c in admin_cols if c in patient_df.columns])

In [31]:
#4 Total bill amount per department
dept_bill_total = (
    patient_billing_df
    .groupby('Department', as_index=False)['BillAmount']
    .sum()
)
print("\nTotal Bill Amount per Department:")
print(dept_bill_total)


Total Bill Amount per Department:
    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


In [32]:
#5 Remove duplicate patient records based on PatientID.
patient_df_nodup = patient_df_cleaned.drop_duplicates(subset='PatientID')

In [36]:
#6 Fill missing BillAmount values with the mean bill amount.
mean_bill = patient_df_nodup['BillAmount'].mean()
patient_df_nodup['BillAmount'] = patient_df_nodup['BillAmount'].fillna(mean_bill)

In [37]:
#7Merge the billing dataset with patient dataset on PatientID.
merged_df = pd.merge(
    patient_df_nodup,
    billing_df,
    on='PatientID',
    how='left'
)
merged_df.info

<bound method DataFrame.info of    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              2000   
1        102      Bob    Neurology   Dr. John  6233.333333              1500   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              2500   
3        104    David   Cardiology  Dr. Smith  6200.000000              3000   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  >

In [38]:
#8 Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
new_patients = pd.DataFrame({
    'PatientID': [9001, 9002],'Department': ['Cardiology', 'Orthopedics'],
    'Doctor': ['Dr. Kumar', 'Dr. Rao'],
    'BillAmount': [12000, 18000]
})
final_df = pd.concat([merged_df, new_patients], axis=0, ignore_index=True)
new_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PatientID   2 non-null      int64 
 1   Department  2 non-null      object
 2   Doctor      2 non-null      object
 3   BillAmount  2 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 196.0+ bytes


In [41]:
#9 Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).
billing_extra = pd.DataFrame({
    'InsuranceCovered': [True] * len(final_df),
    'FinalAmount': final_df['BillAmount'] * 0.9
})

final_df = pd.concat([final_df.reset_index(drop=True), billing_extra], axis=1)
billing_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   InsuranceCovered  7 non-null      bool   
 1   FinalAmount       7 non-null      float64
dtypes: bool(1), float64(1)
memory usage: 195.0 bytes


In [40]:
#final output
print("\nFinal cleaned dataset:")
print(final_df.head())

print("\nFinal dataset info:")
final_df.info()


Final cleaned dataset:
   PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000            2000.0   
1        102      Bob    Neurology   Dr. John  6233.333333            1500.0   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333            1000.0   

   FinalAmount  InsuranceCovered  FinalAmount  
0       3000.0              True       4500.0  
1       3500.0              True       5610.0  
2       5000.0              True       6750.0  
3       3200.0              True       5580.0  
4       4000.0              True       5610.0  

Final dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  